In [2]:
import torch, evaluate, numpy as np, wandb, random, time, os
from sklearn.model_selection import KFold
from datasets import load_dataset
from transformers import AutoImageProcessor, AutoModelForImageClassification, TrainingArguments, Trainer
from torchvision.transforms import (
    Compose,

    RandomVerticalFlip,
    RandomHorizontalFlip,
    RandomResizedCrop,

    CenterCrop,
    Resize,
    Normalize,
    ToTensor,
)

In [3]:
#Parameters
holdout_size = 0.2
epoch = 10
batch_size = 32
lr = 5e-5
n_splits = 5

In [4]:
#Dataset Preprocessing
ds = load_dataset("imagefolder", data_dir="./dataset_24")

kf = KFold(n_splits=n_splits, shuffle=True, random_state=42)

labels1 = ds["train"].features["label"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels1):
    label2id[label] = i
    id2label[i] = label

In [5]:
#Setting evaluation metrics
accuracy = evaluate.load("accuracy")
precision = evaluate.load("precision")
recall = evaluate.load("recall")
f1 = evaluate.load("f1")

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""

    labels = eval_pred.label_ids
    predictions = np.argmax(eval_pred.predictions, axis=1)

    accuracy_results    = accuracy.compute(predictions=predictions, references=labels)
    f1_results          = f1.compute(predictions=predictions, references=labels, average="weighted")
    precision_results   = precision.compute(predictions=predictions, references=labels, average="weighted")
    recall_results      = recall.compute(predictions=predictions, references=labels, average="weighted")
    combined_results    = {**accuracy_results, **f1_results, **precision_results, **recall_results}

    wandb.log({"Train_f1": f1_results["f1"], "Train_prec": precision_results["precision"], "Train_recall": recall_results["recall"]})
    return combined_results

def collate_fn(examples):
    pixel_values = torch.stack([example["pixel_values"] for example in examples])
    labels = torch.tensor([example["label"] for example in examples])
    return {"pixel_values": pixel_values, "labels": labels}

In [6]:
# split up training into training + validation and evaluation

#train_ds = ds

seed = 38218
splits = ds["train"].train_test_split(test_size=holdout_size, stratify_by_column="label", seed=seed)
train_ds = splits['train']
eval_ds = splits['test']

# splits = ds["train"].train_test_split(test_size=testset_size, stratify_by_column="label", seed=seed)
# train_ds = splits['train']
# val_ds = splits['test']

# splits2 = val_ds.train_test_split(test_size=0.5, stratify_by_column="label", seed=seed)
# val_ds = splits2['train']
# eval_ds = splits2['test']

# train_ds.set_transform(preprocess_train)
# val_ds.set_transform(preprocess_val) 
# eval_ds.set_transform(preprocess_val)

In [7]:
for fold, (train_idx, val_idx) in enumerate(kf.split(train_ds)):
    print(f"\nTraining fold {fold + 1}/{n_splits}\n")

    #Setting the Model
    model_checkpoint = "google/vit-base-patch16-224"
    image_processor = AutoImageProcessor.from_pretrained(model_checkpoint)

    normalize = Normalize(mean=image_processor.image_mean, std=image_processor.image_std)
    if "height" in image_processor.size:
        size = (image_processor.size["height"], image_processor.size["width"])
        crop_size = size
        max_size = None
    elif "shortest_edge" in image_processor.size:
        size = image_processor.size["shortest_edge"]
        crop_size = (size, size)
        max_size = image_processor.size.get("longest_edge")

    name = f"3009_{len(id2label)}L_{epoch}E_{batch_size}B_{lr}_{holdout_size}"
    model_name = model_checkpoint.split("/")[-1]
    run_name = f"{model_name}-{name}-{fold}"

    train_transforms = Compose(
        [
            RandomVerticalFlip(),
            RandomHorizontalFlip(),
            RandomResizedCrop(crop_size),
            ToTensor(),
            normalize,
        ]
    )

    val_transforms = Compose(
            [
                Resize(size),
                CenterCrop(crop_size),
                ToTensor(),
                normalize,
            ]
        )

    #Loading Pre-trained Model
    model = AutoModelForImageClassification.from_pretrained(
        model_checkpoint, 
        label2id=label2id,
        id2label=id2label,
        ignore_mismatched_sizes = True, # provide this in case you're planning to fine-tune an already fine-tuned checkpoint
    )

    #Setting training args
    args = TrainingArguments(
        run_name,
        remove_unused_columns=          False,
        evaluation_strategy=            "epoch",
        save_strategy=                  "epoch",
        learning_rate=                  lr,
        per_device_train_batch_size=    batch_size,
        gradient_accumulation_steps=    4,
        per_device_eval_batch_size=     batch_size,
        num_train_epochs=               epoch,
        warmup_ratio=                   0.1,
        log_level=                      "info",
        logging_steps=                  1,
        logging_strategy=               "epoch",
        load_best_model_at_end=         True,
        save_total_limit=               1, 
        metric_for_best_model=          "accuracy",
        report_to=                      "wandb"
    )

    #Split and preprocess data
    train_subset = torch.utils.data.Subset(train_ds, train_idx)
    val_subset = torch.utils.data.Subset(train_ds, val_idx)

    def preprocess_train(example_batch):
        """Apply train_transforms across a batch."""
        example_batch["pixel_values"] = [
            train_transforms(image.convert("RGB")) for image in example_batch["image"]
        ]
        return example_batch

    def preprocess_val(example_batch):
        """Apply val_transforms across a batch."""
        example_batch["pixel_values"] = [val_transforms(image.convert("RGB")) for image in example_batch["image"]]
        return example_batch 

    train_subset.dataset.set_transform(preprocess_train)
    val_subset.dataset.set_transform(preprocess_val) 
    eval_ds.set_transform(preprocess_val)

    #Setting the trainer of model, args, dataset and functions
    trainer = Trainer(
        model,
        args,
        train_dataset=train_subset,
        eval_dataset=val_subset,
        compute_metrics=compute_metrics,
        data_collator=collate_fn,
    )

    wandb.init(project="3009", 
            name=run_name,
            config={
                "Model": model_name,
                "Version": name,
                "Labels": id2label,
                "Epochs": epoch,
                "Batch_size": batch_size,
                "Learning_Rate": lr
            })

    #Training and Evaluation of model
    train_results = trainer.train()
    trainer.save_model()
    trainer.log_metrics("train", train_results.metrics)
    trainer.save_metrics("train", train_results.metrics)

    final = trainer.predict(test_dataset=eval_ds)
    trainer.log_metrics("eval", final.metrics)
    trainer.save_metrics("eval", final.metrics)

    final_labels = final.label_ids
    final_predictions = np.argmax(final.predictions, axis=1)
    wandb.log({f"FinalPred_ConfMatrix_{name}" : wandb.plot.confusion_matrix( 
                preds=final_predictions, y_true=final_labels,
                class_names=id2label
        )})
    wandb.log({"Test_Acc": final.metrics['test_accuracy'], 
            "Test_F1": final.metrics['test_f1'],
            "Test_Prec": final.metrics['test_precision'],
            "Test_Recall": final.metrics['test_recall'],
            "Test_Loss": final.metrics['test_loss']})
    wandb.finish(exit_code=0)


Training fold 1/5



d:\Launchers\Python\lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of ViTForImageClassification were not initialized from the model checkpoint at google/vit-base-patch16-224 and are newly initialized because the shapes did not match:
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([4]) in the model instantiated
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([4, 768]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
d:\Launchers\Python\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `ev

***** Running training *****
  Num examples = 307
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 128
  Gradient Accumulation steps = 4
  Total optimization steps = 20
  Number of trainable parameters = 85,801,732
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.
  0%|          | 0/20 [00:00<?, ?it/s]d:\Launchers\Python\lib\site-packages\torch\optim\optimizer.py:243: UserWarning: 'has_cuda' is deprecated, please use 'torch.backends.cuda.is_built()'
  if not is_compiling() and torch.has_cuda and torch.cuda.is_available():
 10%|█         | 2/20 [00:11<01:23,  4.66s/it]***** Running Evaluation *****
  Num examples = 77
  Batch size = 32


{'loss': 1.9254, 'grad_norm': 7.22806978225708, 'learning_rate': 5e-05, 'epoch': 0.8}


                                              
 10%|█         | 2/20 [00:12<01:23,  4.66s/it]Saving model checkpoint to vit-base-patch16-224-3009_4L_10E_32B_5e-05_0.2-0\checkpoint-2
Configuration saved in vit-base-patch16-224-3009_4L_10E_32B_5e-05_0.2-0\checkpoint-2\config.json


...

100%|██████████| 20/20 [01:21<00:00,  4.07s/it]
Saving model checkpoint to vit-base-patch16-224-3009_4L_10E_32B_5e-05_0.2-4
Configuration saved in vit-base-patch16-224-3009_4L_10E_32B_5e-05_0.2-4\config.json
Model weights saved in vit-base-patch16-224-3009_4L_10E_32B_5e-05_0.2-4\model.safetensors
***** Running Prediction *****
  Num examples = 96
  Batch size = 32


***** train metrics *****
  epoch                    =         8.0
  total_flos               = 177830164GF
  train_loss               =      0.6255
  train_runtime            =  0:01:21.08
  train_samples_per_second =      37.984
  train_steps_per_second   =       0.247


100%|██████████| 3/3 [00:00<00:00,  6.37it/s]


***** eval metrics *****
  test_accuracy           =     0.8333
  test_f1                 =     0.8244
  test_loss               =     0.5501
  test_precision          =     0.8405
  test_recall             =     0.8333
  test_runtime            = 0:00:01.05
  test_samples_per_second =     91.395
  test_steps_per_second   =      2.856


Test_Acc,▁
Test_F1,▁
Test_Loss,▁
Test_Prec,▁
Test_Recall,▁
Train_f1,▁▅▇▇████▇
Train_prec,▁▆█▇████▇
Train_recall,▁▄▇▇████▇
eval/accuracy,▁▄▇▇████
eval/f1,▁▅▇▇████
eval/loss,█▆▄▂▂▁▁▁
